### Selection of data from 2019 with at least 3 remote sensors, 1,2,3 or 4 occupants and from Florida or California

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

seed=2020

In [2]:
# Import the data
df1=pd.read_csv('F:/metadata/meta_data.csv')
# Remove repeating rows and unrepresentative data
df = df1.drop_duplicates()
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]
# Selection of useful data
df=df[(df['Country']=='US') & ((df['ProvinceState']=='CA') | (df['ProvinceState']=='FL'))]
df=df[df['Number of Remote Sensors'] >=3]
df=df[(df['Number of Occupants']<=4) &  (df['Number of Occupants'] >0)]
df.shape

(1412, 22)

In [3]:
# List all 2019 files in directory using pathlib
lista = []
basepath = Path("/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)

In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista] 
df_in_2019 = df[df['Identifier'].isin(lista_FileName)]

In [ ]:
group_df = df_in_2019[['Identifier','ProvinceState']].groupby('ProvinceState').count()
ax_ProvinceState= group_df.plot(kind='bar',figsize=(9, 6))
plt.xticks(rotation=0)
ax_ProvinceState.set_title("Number of identifiers in each province in 2019 with at least 3 sensors")
ax_ProvinceState.set_xlabel('ProvinceState')
ax_ProvinceState.set_ylabel('Number of Identifiers')

rects_Country = ax_ProvinceState.patches # ubicacion de las barras del grafico 

y_value_Country = group_df['Identifier']
labels_Country = y_value_Country # valores de altura de cada barra(valor en y)

for rect, label in zip(rects_Country, labels_Country): # uno valor y ubicacion de cada barra para asignarselos uno a uno en el grafico con ax.text
       height = rect.get_height()
       ax_ProvinceState.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

In [6]:
# Resample dataframe
resample_value = df_in_2019['ProvinceState'].value_counts().min()
df_province1 = df_in_2019[df_in_2019['ProvinceState']=='CA'].sample(resample_value,random_state=seed)
df_province2 = df_in_2019[df_in_2019['ProvinceState']=='FL'].sample(resample_value,random_state=seed)

df_province_1_2 = pd.concat([
    df_province1,
    df_province2
])

In [7]:
# 
'''"fig, axes = plt.subplots()

x = np.arange(start=0, stop=20, step=5)  # the label locations
width = 0.4

axes.bar(x-width/2, df_province1['Number of Occupants'].values,width)
axes.bar(x+width/2, df_province2['Number of Occupants'].values,width)
df_province1.shape"'''

'"fig, axes = plt.subplots()\n\nx = np.arange(start=0, stop=20, step=5)  # the label locations\nwidth = 0.4\n\naxes.bar(x-width/2, df_province1[\'Number of Occupants\'].values,width)\naxes.bar(x+width/2, df_province2[\'Number of Occupants\'].values,width)\ndf_province1.shape"'

In [9]:
use_cols = []
mainPath =
# Take all Identifiers of df_province_1_2, to after open them
a=df_province_1_2.loc[:,'Identifier'].tolist()
lista=[]
for i in a:
    lista.append(mainPath+i+'.csv')

In [10]:
# Save the data to open it with dask
df_province_1_2.to_csv('sv') 

In [11]:
#Open the data with Dask
dd_province_1_2 = dd.read_csv('sv').drop('Unnamed: 0',axis=1)

In [12]:
# Read all files in the df_province_1_2['Identifier'] column
ff=dd.read_csv(lista, usecols = use_cols, parse_dates=['DateTime'], include_path_column=True)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')

In [13]:
# Manage of columns
ff['activation'] = ff.loc[:, ff.columns.str.contains('_Motion')].sum(axis=1) # Sum of the number of RS in 1 in each row
ff['dayofweek']=ff['DateTime'].dt.day_name()# Know what day of the week it is
ff['Time'] = ff['DateTime'].dt.time
#ff['Month'] = ff['DateTime'].dt.month

ff_columns = ['path', 'activation', 'dayofweek', 'Time']#, 'Month']
result = dd.merge(dd_province_1_2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result['mean_activation']=result['activation']/(result['Number of Remote Sensors']+1) # Normalization according to the number of sensors that each identifier has (+1 counting sensor of thermostat)

In [14]:
# Group by identifier, time and day of the week
aa=result[['Identifier','dayofweek','Time','mean_activation']].groupby(['Identifier','Time','dayofweek']).mean()
pp=dd.from_array(aa.to_records()) 

In [15]:
# concatenating Time with dayofweek 
pp["TimeSTR"]=pp["Time"].apply((lambda x: x.strftime('%H:%M')), meta='str')
aa2 = pp.copy()
new = aa2["TimeSTR"]
# overwriting name column 
aa2["dayofweek"]= aa2["dayofweek"].str.cat(new, sep =", ") 

In [16]:
# Rotating the table to have the time-day of the week as columns
bb=aa2.drop(['Time'],axis=1)
bb = bb.categorize(columns='dayofweek')
table = dd.pivot_table(bb, values='mean_activation', index='Identifier',
                    columns='dayofweek')

In [17]:
t=dd.from_array(table.to_records())
result_f = dd.merge(dd_province_1_2,t, on='Identifier').drop(['filename'],axis=1) # Join to have the complete data of each row

In [18]:
#(result_f.compute()).to_csv('.csv')